<a href="https://colab.research.google.com/github/JHyunjun/torch_GAN_AutoEncoder/blob/main/LSTM_GAN_AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy import array
import pandas as pd
import numpy as np
import pandas_datareader.data as pdr
import matplotlib.pyplot as plt
import datetime
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#Data Load
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Batt/train_v.txt') 
#df_train = df_train.astype(int)
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Batt/test_v.txt')
train_data_input = df_train.iloc[:,:-1]
train_data_output = df_train.iloc[:,-1]
test_data_input = df_test.iloc[:,:-1]
test_data_output = df_test.iloc[:,-1]

train_data = df_train.iloc[:,:]
test_data = df_test.iloc[:,:]

trainSet_for_result = train_data
testSet_for_result = test_data

print("train data input : ",train_data_input.shape,train_data_input)
print("train data output : ",train_data_output.shape, train_data_output)
print("train_data : ",train_data.shape,train_data)
print("test_data : ",test_data.shape,test_data)

In [ ]:
time1 = 0
time2 = 60
train_gan = df_train.iloc[time1:time2, :].values
print(train_gan.shape)

train_gan2 = df_test.iloc[time1:time2, :].values
#train_gan2 = train_gan2.to_numpy()
print(train_gan)

In [ ]:
#Normalization

def origin_minmax(data) : 
  arr_max = np.zeros(data.shape[1])
  arr_min = np.zeros(data.shape[1])
  for i in range(data.shape[1]) : 
    max = np.max(data[:,i])
    min = np.min(data[:,i]) 
    arr_max[i] = max
    arr_min[i] = min
    pass
  return arr_max, arr_min

arr_max1, arr_min1 = origin_minmax(train_gan)
arr_max2, arr_min2 = origin_minmax(train_gan2)

def MinMaxScaler(data):
  for i in range(data.shape[1]) : 
    max = np.max(data[:,i])
    min = np.min(data[:,i])
    data[:,i] = (data[:,i] - min) / (max - min + 1e-7)
    pass
  return data

train_gan = MinMaxScaler(train_gan)
train_gan2 = MinMaxScaler(train_gan2)

def back_MinMax(data,max,min):
  for i in range(data.shape[1]) : 
    data[:,i] = data[:,i] * (max[i] - min[i]) + min[i]
    pass
  return data

#train_gan = torch.tensor([train_gan], dtype = torch.float32)
#train_gan2 = torch.tensor([train_gan2], dtype = torch.float32)
print(train_gan.shape)

print(arr_max1)
print(arr_min1)
print(arr_max2)
print(arr_min2)

In [ ]:
# Windowing for Sequence

seq_length = 3

def sliding_window(time_series, seq_length):
    time_series = np.squeeze(time_series)
    dataX = []
    dataY = []
    for i in range(0, len(time_series) - seq_length): 
        _x = time_series[i:i + seq_length, :]
        _y = time_series[i:i + seq_length, :]  # for AE
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

train_gan_window_X, train_gan_window_Y = sliding_window(train_gan, seq_length)
test_gan_window_X, train_gan_window_Y = sliding_window(train_gan2, seq_length)

train_gan_window_X = torch.tensor(train_gan_window_X)
print(train_gan_window_X.shape)
print(train_gan_window_Y.shape)

train_gan_window_X = torch.tensor(train_gan_window_X, dtype = torch.float32)
train_gan_window_Y = torch.tensor(train_gan_window_Y, dtype = torch.float32)

In [ ]:
hidden_dim_D = 10

class Discriminator(nn.Module):
    
    def __init__(self, input_dim = seq_length, n_features = train_gan_window_X.shape[2]):
        # initialise parent pytorch class
        super().__init__()
        self.seq_len, self.input_dim = seq_length, input_dim
        self.hidden_dim, self.n_features = hidden_dim_D, n_features
        self.a = nn.LSTM(input_size = n_features, hidden_size = self.hidden_dim , num_layers = 1, batch_first = True, bidirectional = False, dropout = 0)
        self.b = nn.LeakyReLU(0.02)

        self.c = nn.LSTM(input_size = self.hidden_dim, hidden_size = self.hidden_dim , num_layers = 1, batch_first = True, bidirectional = False, dropout = 0)
        self.d = nn.LeakyReLU(0.02)
        #self.e = nn.LayerNorm(Data_seq * dim_size)

        self.f = nn.Linear(self.hidden_dim, self.hidden_dim) 
        self.f_1 = nn.Linear(self.hidden_dim * seq_length, 1)
        self.g = nn.Sigmoid()
        
        # create loss function
        #W-GAN
        #self.loss_function = nn.BCELoss()
        self.loss_function = nn.MSELoss()

        # create optimiser, simple stochastic gradient descent
        self.optimiser = torch.optim.Adam(self.parameters(), lr=0.0001)
        # Wasserstain
        #self.optimiser = torch.optim.RMSprop(self.parameters(), lr = 5e-5)

        # counter and accumulator for progress
        self.counter = 0;
        self.progress = []
        pass
    
    def forward(self, inputs):
        #simply run model
        #print("input : ", inputs.shape)
        x, (_, _) = self.a(inputs)
        #print("A", x.shape)
        x = self.b(x)
        #print("B", x.shape)
        x, (_, _) = self.c(x)
        #print("C", x.shape)
        x = self.d(x)
        #print("D", x.shape)
        x = self.f(x)
        x = torch.flatten(x)
        #print("F", x.shape)
        x = self.f_1(x)
        #print("F_1 : ",x.shape)

        x = self.g(x)
        #print("G", x.shape)

        return x

    def train(self, inputs, targets, gens):
        # calculate the output of the network
        #print("Start")
        outputs = self.forward(inputs)
        #print("After D_self.forward(inputs)")
        targets_w = self.forward(gens)
        #print("After self.forward(gens)")

        # calculate loss
        loss = self.loss_function(outputs, targets)
        #print("After Calculating D loss")

        # Wasserstein loss
        #loss = -(torch.mean(outputs) - torch.mean(targets_w))

        # increase counter and accumulate error every 10
        self.counter += 1;
        if (self.counter % 10 == 0):
            self.progress.append(loss.item())
            pass
        if (self.counter % 1000 == 0):
            print("counter = ", self.counter)
            pass

        # zero gradients, perform a backward pass, update weights
        self.optimiser.zero_grad()
        loss.backward()
        self.optimiser.step()
        
        #W-GAN
        #for p in self.parameters() : 
        #  p.data.clamp_(-0.01,0.01)
        #pass
    
    def plot_progress(self):
        df = pd.DataFrame(self.progress, columns=['Discriminator loss'])
        df.plot(ylim=(0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0, 0.25, 0.5, 1.0, 5.0))
        pass
    
    pass

In [ ]:
#Training the Discriminator
N = 10  #iterations

D = Discriminator()
targets = torch.FloatTensor([1.0])
non_target = torch.FloatTensor([0.0])

In [ ]:
#Checking the performance of Discriminator
#D.plot_progress()


class View(nn.Module):
    def __init__(self, shape):
        super().__init__()
        self.shape = shape,

    def forward(self, x):
        return x.view(*self.shape)


In [ ]:
# Constructing Generator for AutoEncoder Structure

hidden_dim = 20

class Generator(nn.Module) : 
  def __init__(self, input_dim = seq_length, n_features = train_gan_window_X.shape[2]) : 
    super().__init__()
    self.seq_len, self.input_dim = seq_length, input_dim
    self.hidden_dim, self.n_features = hidden_dim, n_features
    self.a = nn.LSTM(input_size = n_features, hidden_size = self.hidden_dim * 2, num_layers = 1, batch_first = True, bidirectional = False, dropout = 0)
    self.b = nn.LeakyReLU(0.02)
    self.c = nn.LSTM(input_size = hidden_dim * 2, hidden_size = self.hidden_dim, num_layers = 1, batch_first = True, bidirectional = False, dropout = 0) 
    self.d = nn.LeakyReLU(0.02)
    self.e = nn.LSTM(input_size = hidden_dim, hidden_size = self.hidden_dim * 2, num_layers = 1, batch_first = True, bidirectional = False, dropout = 0) 
    self.f = nn.Linear(self.hidden_dim * 2, features)
    self.g = nn.Sigmoid()
    #View((1, train_gan.shape[1], 1))   

    self.optimiser = torch.optim.Adam(self.parameters(), lr = 0.0001)
    #Wasserstain
    #self.optimiser = torch.optim.RMSprop(self.parameters(), lr = 5e-5)
    self.counter = 0
    self.progress = []

    pass

  def forward(self, inputs) : 
    #print("inputs. shape : ",inputs.shape) #numpyarray ([seq_length, features])
    x, (_, _) = self.a(inputs)
    #print("a. shape : ",x.shape)
    x = self.b(x)
    #print("b. shape : ",x.shape)
    x, (_, _) = self.c(x)
    #print("c. shape : ",x.shape)
    x = self.d(x)
    #print("d. shape : ",x.shape)
    x, (_, _) = self.e(x) 
    #print("e. shape : ",x.shape)
    x = self.f(x) 
    #print("f. shape : ",x.shape)
    x = self.g(x)
    #print("g. shape : ",x.shape)
    x = x.reshape(seq_length , features) # Should be [seq_length, features]
    #print("h. shape : ",x.shape)
    return x

  def train(self, D, inputs, targets) : 
    g_output = self.forward(inputs)
    d_output = D.forward(g_output)
    #loss = D.forward(g_output)
    loss = D.loss_function(d_output, targets)

    #Wasserstain
    #loss = -torch.mean(d_output)

    self.counter+=1;
    if (self.counter % 10 == 0) :
      self.progress.append(loss.item())
      pass

    self.optimiser.zero_grad()
    loss.backward()
    self.optimiser.step()

  def plot_progress(self):
        df = pd.DataFrame(self.progress, columns=['Generator loss'])
        df.plot(ylim=(0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0, 0.25, 0.5, 1.0, 5.0))
        
  pass

In [ ]:
ref = train_gan_window_X #Normalizing된 신호
print(ref.shape)


In [ ]:
#Training both Generator and Discriminator for pattern

features = ref.shape[2] #5, seq_length = 3

D = Discriminator()
G = Generator()

for i in range(time2-time1-seq_length) : #i는 0부터 시작
  refs = train_gan_window_X[i, :, :] #[897, 3(seq_length), 5(features)]
  #print(refs.shape) #[3(seq_length), 5(features)]
  for j in range(1) : 
    D.train(refs, torch.FloatTensor([1.0]),G.forward(refs))
    G.train(D, refs, torch.FloatTensor([1.0]))
  if(i%100 == 0) :  print("i iteration : ", i)

  pass
pass


In [ ]:
#Plotting the Generator Loss
D.plot_progress()
G.plot_progress()
print(len(train_gan)) 

In [ ]:
for i in range(time2 - time1 - seq_length) : 
  seed = train_gan_window_X[i, :, :]
  print("Real Value : ", seed)
  out = G.forward(seed)
  out = out.detach().cpu().numpy()
  print("Generated Value : ", out)


In [ ]:
for i in range(time2-time1) : 
  seed = train_gan[:,:, i:(i+1)]
  seed = np.squeeze(seed)
  out = G.forward(seed)
  out = out.detach().cpu().numpy()
  #print(out.shape)
  out = np.reshape(out, [1,train_gan.shape[1]])
  seed = np.reshape(seed, [1,train_gan.shape[1]])
  #print(out.shape)
  out_dim = back_MinMax(out, arr_max1, arr_min1)
  print("Generated Value : ",i," = ", out_dim)
  real_value = back_MinMax(seed, arr_max1, arr_min1)
  print("Real Value : ",i, " = ",real_value )
  print("                            ")